<a href="https://colab.research.google.com/github/mateusrodriguesq/avaliacao-supera/blob/main/processo_seletivo_supera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Avaliação Processo Seletivo — Supera, Analista de dados Pleno**

Bom dia, meu nome é Mateus Rodrigues e realizarei um teste para o processo seletivo da Quaest para avaliar minhas habilidades como Analista de dados por meio da linguagem de programação Python via Google Colab.

A atividade é analisar dois bancos de dados e propor um modelo onde preverá se o filme irá passar ou não no teste de Bechdel.

#**Banco de dados e livrarias**


Primeiramente, é essencial importarmos o banco de dados para o Google Colab. Por ser uma plataforma em nuvem podemos fazê-lo de várias formas. Uma delas, para banco de dados não tão extensos em tamanho de arquivo, como este(35kbs), se torna mais fácil "*dar upload*" no arquivo já disponibilizado pelo email.

In [389]:
#Importarei uma livraria do próprio Google Colab chamada "files" com propósito de conseguirmos colocar o arquivo e instanciá-lo em um *DataFrame*
from google.colab import files
upload = files.upload()

In [390]:
#Importar o banco de dados de teste: test.csv
upload = files.upload()

Outra biblioteca muito importante que iremos adicionar antes mesmo de começarmos é a livraria **pandas**, que sem dúvidas é a mais utilizada em todo meio de manipulação de banco de dados dentro de arquivos em Python, conseguindo lidar com qualquer tipo de arquivo (.csv, .xls, .xlsx, .json e afins). 

In [391]:
#Primeiro importaremos. Para facilitar em toda a análise, será simplificado seu nome utilizando a palavra-chave "as" para transformar o "pandas" em "pd", como um apelido.
import pandas as pd
#Livraria para manipulações algébricas
import numpy as np
#Livraria para LabelEncoder(transformando variáveis categóricas em  numéricas)
from sklearn import preprocessing
#Visualização interativa de dados
import plotly.graph_objects as go
import plotly.offline as py
import plotly.express as px

#Declarando os bancos de dados em variáveis
bechdel = 'bechdel.xlsx'
test = 'test.csv'
#Utilizando a função pd.read_csv para "test.csv" e pd.read_excel para "bechdel.xlsx"
teste = pd.read_csv(test)
bh_df = pd.read_excel(bechdel,sheet_name=0)
bh_df.head()

,Year,IMDB code,Budget,Dom gross,Int Gross,Code,budget 2013 dollars,Dom gross 2013 dollars,Int gross 2013 dollars,Simple pass / fail,Unnamed: 10,Unnamed: 11
0,year,imdb,budget,domgross,intgross,code,budget_2013,domgross_2013,intgross_2013,binary,NaN,NaN
1,1974,tt0071562,13000000,57300000,57300000,1974PASS,61408439,270669505,270669505,PASS,NaN,NaN
2,1982,tt0084516,10700000,74706019,121706019,1982PASS,25821968,180285645,293709241,PASS,NaN,NaN
3,2008,tt0800241,15000000,2203641,6379575,2008PASS,16233845,2384904,6904335,PASS,NaN,https://en.wikipedia.org/wiki/Bechdel_test
4,2011,tt1625346,12000000,16311571,22750356,2011PASS,12428289,16893744,23562334,PASS,NaN,NaN


In [392]:
teste.head()

,year,imdb,budget,domgross,intgross,budget_2013,domgross_2013,intgross_2013
0,2013,tt1559547,50000000,19452138.0,55940671.0,50000000,19452138.0,55940671.0
1,2013,tt0765446,40000000,57012977.0,66454811.0,40000000,57012977.0,66454811.0
2,2013,tt1211956,70000000,25213103.0,103813103.0,70000000,25213103.0,103813103.0
3,2013,tt1351685,195000000,65187603.0,197387603.0,195000000,65187603.0,197387603.0
4,2013,tt1650554,28000000,28795985.0,60839197.0,28000000,28795985.0,60839197.0


# **Esclarecimento das Atividades**


O desafio consiste em 3 partes: 

* **1ª etapa, a "configuração de dados":** Importar o *dataset* chamado *"Bechdel.xlsx"* em Python ou R e preparar o banco de dados para a análise.

Aprender sobre o teste de Bechdel e ver o *data dictionary* para detalhes dos nomes das variáveis.

* **2ª etapa, Análise de dados:** Resumir os dados utilizando métodos estatísticos respondendo duas questões:

> Qual a porcentagem dos filmes que passaram no teste, baseado na definição binária?

> Como a porcentagem de filmes que passaram mudaram com o tempo?

> Propor um modelo que irá prever se o filme passa ou falha no teste, baseada em qualquer das variáveis que forem úteis.

> Utilizar o banco de dados chamado *test.csv*, aplicando o modelo preditivo, classificando este banco de dados se estes novos filmes irão passar ou não no teste(1=passou, 0=falhou). Salvar as predições num arquivo chamado *mdb.csv*.

* **3ª etapa, Controle de versão:** Disponibilizar o código em um repositório público no Github ou serviços similares. Prover o link



 







#**1ª Etapa**

##**Limpeza de Dados**

A nossa função na limpeza de dados é facilitar o processo de **Análise Exploratória de dados**, seja esta análise por tabelas ou gráficos. Ás vezes numa coleta de dados, nem sempre é otimizada o suficiente para obtermos uma análise limpa, como nomes resumidos ou mal compreendidos, colunas com muitas informações, colunas não tão importantes e afins. Por isto, iremos descomplicar certas atividades realizadas no futuro.

###**Primeira modificação**
 Remover duas colunas com valores nulos e em outras colunas.
Como as colunas não acrescentam nada além do link para o Teste de Bechdel, onde já foi obtido nas orientações dentro do arquivo no email, iremos retirá-las e vamos analisar se há valores nulos em outras variáveis e removê-los.


In [393]:
#Removendo as colunas não nomeadas 10 e 11.
bh_df = bh_df.drop(columns=bh_df.columns[11])
bh_df = bh_df.drop(columns=bh_df.columns[10])
bh_df.head()

,Year,IMDB code,Budget,Dom gross,Int Gross,Code,budget 2013 dollars,Dom gross 2013 dollars,Int gross 2013 dollars,Simple pass / fail
0,year,imdb,budget,domgross,intgross,code,budget_2013,domgross_2013,intgross_2013,binary
1,1974,tt0071562,13000000,57300000,57300000,1974PASS,61408439,270669505,270669505,PASS
2,1982,tt0084516,10700000,74706019,121706019,1982PASS,25821968,180285645,293709241,PASS
3,2008,tt0800241,15000000,2203641,6379575,2008PASS,16233845,2384904,6904335,PASS
4,2011,tt1625346,12000000,16311571,22750356,2011PASS,12428289,16893744,23562334,PASS


In [394]:
#Verificando se há valores nulos.
bh_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1501 entries, 0 to 1500
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Year                    1501 non-null   object
 1   IMDB code               1501 non-null   object
 2   Budget                  1501 non-null   object
 3   Dom gross               1486 non-null   object
 4   Int Gross               1492 non-null   object
 5   Code                    1501 non-null   object
 6   budget 2013 dollars     1501 non-null   object
 7   Dom gross 2013 dollars  1485 non-null   object
 8   Int gross 2013 dollars  1492 non-null   object
 9   Simple pass / fail      1501 non-null   object
dtypes: object(10)
memory usage: 117.4+ KB


In [395]:
#Podemos observar que há alguns valores nulos nas colunas: Dom gross, Int Gross, Dom Gross 2013 dollars e Int Gross 2013 dollars
#Pela pequena quantidade, será retirado as linhas dos valores nulos.
bh_df = bh_df[bh_df['Dom gross'].notna()]
bh_df = bh_df[bh_df['Int Gross'].notna()] 
bh_df = bh_df[bh_df['Dom gross 2013 dollars'].notna()] 
bh_df = bh_df[bh_df['Int gross 2013 dollars'].notna()]
bh_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1485 entries, 0 to 1500
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Year                    1485 non-null   object
 1   IMDB code               1485 non-null   object
 2   Budget                  1485 non-null   object
 3   Dom gross               1485 non-null   object
 4   Int Gross               1485 non-null   object
 5   Code                    1485 non-null   object
 6   budget 2013 dollars     1485 non-null   object
 7   Dom gross 2013 dollars  1485 non-null   object
 8   Int gross 2013 dollars  1485 non-null   object
 9   Simple pass / fail      1485 non-null   object
dtypes: object(10)
memory usage: 127.6+ KB


In [396]:
bh_df.head()

,Year,IMDB code,Budget,Dom gross,Int Gross,Code,budget 2013 dollars,Dom gross 2013 dollars,Int gross 2013 dollars,Simple pass / fail
0,year,imdb,budget,domgross,intgross,code,budget_2013,domgross_2013,intgross_2013,binary
1,1974,tt0071562,13000000,57300000,57300000,1974PASS,61408439,270669505,270669505,PASS
2,1982,tt0084516,10700000,74706019,121706019,1982PASS,25821968,180285645,293709241,PASS
3,2008,tt0800241,15000000,2203641,6379575,2008PASS,16233845,2384904,6904335,PASS
4,2011,tt1625346,12000000,16311571,22750356,2011PASS,12428289,16893744,23562334,PASS


### **Segunda modificação**
  Renomear nomes extensos das colunas e remover a linha 0.

Renomearemos o nome das colunas pelos nomes da linha 0, pela extensão desnecessária dos nomes.

In [397]:
#Renomeando as colunas com a linha 0.
bh_df.columns = bh_df.iloc[0]
#Removendo a linha 0.
bh_df = bh_df.drop([0], axis=0)

In [398]:
bh_df.head()

,year,imdb,budget,domgross,intgross,code,budget_2013,domgross_2013,intgross_2013,binary
1,1974,tt0071562,13000000,57300000,57300000,1974PASS,61408439,270669505,270669505,PASS
2,1982,tt0084516,10700000,74706019,121706019,1982PASS,25821968,180285645,293709241,PASS
3,2008,tt0800241,15000000,2203641,6379575,2008PASS,16233845,2384904,6904335,PASS
4,2011,tt1625346,12000000,16311571,22750356,2011PASS,12428289,16893744,23562334,PASS
5,2000,tt0190590,26000000,45506619,75763814,2000FAIL,35175618,61566286,102501499,FAIL


#**2ª etapa**




Primeiramente, para a resposta de ambas perguntas, como é lidado com dados categóricos na coluna "binary_test", é importante transformar estes valores para númericos, onde 1 é igual ao filme que passou no teste e 0 houve falha. Será usufruido da livraria *preprocessing do Sklearn* chamado **LabelEncoder**.

**Primeira modificação**
 Transformar os valores categóricos da coluna "binary_test" em 0 e 1.


In [399]:
#Instanciando o Label Encoder na variável Label
label = preprocessing.LabelEncoder()
#Aplicando e transformando os valores categóricos da coluna em números.
bh_df['binary'] = label.fit_transform(bh_df.binary.values)
#P.s: Passou no teste = 1, Não passou no teste = 0.
bh_df.head()

,year,imdb,budget,domgross,intgross,code,budget_2013,domgross_2013,intgross_2013,binary
1,1974,tt0071562,13000000,57300000,57300000,1974PASS,61408439,270669505,270669505,1
2,1982,tt0084516,10700000,74706019,121706019,1982PASS,25821968,180285645,293709241,1
3,2008,tt0800241,15000000,2203641,6379575,2008PASS,16233845,2384904,6904335,1
4,2011,tt1625346,12000000,16311571,22750356,2011PASS,12428289,16893744,23562334,1
5,2000,tt0190590,26000000,45506619,75763814,2000FAIL,35175618,61566286,102501499,0


##**i) Resumo dos dados e perguntas**

###**Primeira pergunta**
 Qual a porcentagem dos filmes que passaram no teste, baseado na definição binária?




In [400]:
#Para esta resposta, contaremos todos os valores da coluna e ativaremos o parâmetro "normalize", onde utiliza a porcentagem decimal das categorias numéricas.
bh_df["binary"].value_counts(normalize=True)

0    0.552561
1    0.447439
Name: binary, dtype: float64

Respondendo a primeira pergunta, a porcentagem dos filmes que passaram no teste de Bechdel é de **44.74%**, baseado na definição binária. Já os filmes reprovados é de **55.25%**.

###**Segunda pergunta**
 Como a porcentagem de filmes que passaram mudaram com o tempo? 



In [401]:
#Já para esta resposta, criaremos uma tabela de contingência com o ano de cada filme e sua aprovação ou rejeição no teste
tabela_ano = pd.crosstab(bh_df['year'], bh_df['binary'], margins=False)
tabela_ano_porc = pd.crosstab(bh_df['year'], bh_df['binary'], margins=False, normalize='index')

In [402]:
tabela_ano_porc.head()

binary,0,1
year,,
1970,0.000000,1.000000
1971,1.000000,0.000000
1972,0.500000,0.500000
1973,0.800000,0.200000
1974,0.666667,0.333333


Como uma tabela de contingência, se torna algo muito extenso e nada intuitivo. Com dados extensos classificados assim por ano se torna interessante a aplicação de um Histograma para a visualização de dados. A biblioteca *plotly.express* é de boa utilidade pela documentação gráfica e interativa, pois temos duas classificações no eixo y (Teste Binário = 0 e 1).

In [403]:
#Instanciaremos a função px.histogram em "fig" e utilizando as funções para update a figura, será mudado título, legendas e eixos.
fig = px.histogram(bh_df, x="year", color="binary", title="Gráfico de Barras Interativo - Classificação do Teste Binário entre os anos 1970-2013",
                   labels={"year": "Ano", "binary":"Teste Binário"}).update_xaxes(categoryorder="total descending")
fig.update_xaxes(title_text="Anos")
fig.update_yaxes(title_text="Quantidade de filmes")
fig.show()

Como é possível observar, entre as décadas de 1970 até 1990 o Teste de Bechdel era realmente de frequência bem baixa de se passar, onde somente em 1988 houve um significativa relevância de 47,3%.

 Já a partir de 1995, houve um provável movimento cultural a favor de filmes sem personagens mulheres onde só havia interação quando o assunto era homens, com o crescimento representando 51,6% da cota de filmes anualmente.
 
Onde a era de acréscimo de obras ficcionais serem aprovadas no teste foi entre 1995 até 2004. A partir disso, a maioria dos filmes se manteve maioria falha no teste.

##**ii) Propor modelo para previsão**

Para a criação de um modelo de previsão, é importante a formatação dos banco de dados de treino*("bechdel.xlsx")* e o de teste*("teste.csv")*. Com isso como primeira modificação é a retirada de colunas não importantes para a modelação no banco de dados com os testes binários.


###**Primeira modificação**

Selecionar somente as colunas influentes para o modelo, modificando os tipos de variáveis para cada correspondente e indexando a coluna **"imdb"**.


In [404]:
bh_df.head(1)

,year,imdb,budget,domgross,intgross,code,budget_2013,domgross_2013,intgross_2013,binary
1,1974,tt0071562,13000000,57300000,57300000,1974PASS,61408439,270669505,270669505,1


In [405]:
#Selecionando uma tabela para o treino do modelo com as variáveis "Code" retirada.
treino = bh_df[['year', 'budget', 'domgross', 'intgross', 'budget_2013', 'domgross_2013', 'intgross_2013']]

#Indexando a coluna imbd em ambos dataframes
teste = teste.drop(['imdb'], axis=1)
#teste = teste.set_index('imdb')
#treino = treino.set_index('imdb')
treino

,year,budget,domgross,intgross,budget_2013,domgross_2013,intgross_2013
1,1974,13000000,57300000,57300000,61408439,270669505,270669505
2,1982,10700000,74706019,121706019,25821968,180285645,293709241
3,2008,15000000,2203641,6379575,16233845,2384904,6904335
4,2011,12000000,16311571,22750356,12428289,16893744,23562334
5,2000,26000000,45506619,75763814,35175618,61566286,102501499
...,...,...,...,...,...,...,...
1496,1978,3000000,3446749,3660880,10712118,12307327,13071926
1497,1998,13000000,16757163,16757163,18578720,23948203,23948203
1498,2006,11000000,72778712,85278712,12713760,84117369,98564824
1499,2005,88000000,61649911,108539911,104986915,73550386,129491710


In [406]:
#Verificando o tipo de cada coluna
print(treino.dtypes,'\n________________________\n', teste.dtypes)

0
year             object
budget           object
domgross         object
intgross         object
budget_2013      object
domgross_2013    object
intgross_2013    object
dtype: object 
________________________
 year               int64
budget             int64
domgross         float64
intgross         float64
budget_2013        int64
domgross_2013    float64
intgross_2013    float64
dtype: object


In [407]:
#É observado onde o dataset de teste está bem especificado cada coluna em sua tipificação, com isto iremos aplicar das mesmas no treino.
treino['domgross'] = treino.domgross.astype(float)
treino['intgross'] = treino.intgross.astype(float)
treino['domgross_2013'] = treino.domgross_2013.astype(float)
treino['intgross_2013'] = treino.intgross_2013.astype(float)
treino['budget_2013'] = treino.budget_2013.astype(int)
treino['year'] = treino.year.astype(int)
treino['budget'] = treino.budget.astype(int)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

##**Definição e aplicação do modelo**

Iremos aplicar uma livraria/modelo do Scikit Learn chamado **Random Forest Classifier**. Onde utiliza um mecanismo com várias árvores de decisão, aleatoriamente. Este algoritmo é utilizando em regressão tão quanto classificação.

Pelo fator da aleatoriedade, é de suma importância a aplicação da validação do modelo. E uma das mais conhecidas validações é a cruzada, onde este modelo se destaca fornecendo ótimas precisões.

E para a aplicação da validação cruzada ser eficaz, iremos utilizar o método de **Repeated K-Fold**, onde é um meio para melhorar a perfomance estimada do nosso modelo de *machine learning* utilizando a repetição em k vezes e as aleatorizando em cada uma destas vezes.

In [408]:
# Importando a livraria de Random Forest
from sklearn.ensemble import RandomForestClassifier
#Instanciando em uma variável chamada modelo
modelo = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=0)
#Livraria do método de "Repeated Kfold"
from sklearn.model_selection import RepeatedKFold
#Importando a livraria da divisão entre treino e teste
from sklearn.model_selection import train_test_split

In [410]:
#Chamaremos as variáveis de treino de X e a variável que está sendo prevista(Coluna 'binary') de y
X = treino
y = bh_df[['binary']]
#Ajustando o modelo nos dados de treino
X = X. fillna(-1)
modelo.fit(X, y.values.ravel())

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [437]:
#Instanciando os dados de teste
X_prev = teste
X_prev = X_prev.fillna(-1)
#Aplicando o modelo para prever os dados de teste
h = modelo.predict(X_prev)
pd.Series(h)

0      0
1      1
2      0
3      0
4      0
      ..
289    0
290    0
291    0
292    0
293    0
Length: 294, dtype: int64

In [413]:
#Separando dados de treino e teste do banco de dados de Bechdel
np.random.seed(0)
X_treino, X_valid, y_treino, y_valid = train_test_split(X, y, test_size=0.5)
X_treino.head()

,year,budget,domgross,intgross,budget_2013,domgross_2013,intgross_2013
977,2009,200000000,125322469.0,371628539.0,217146319,136066564.0,403488847.0
523,1986,15000000,179800601.0,356800601.0,31856471,381854174.0,757760530.0
1378,2012,275000000,73058679.0,282778100.0,279025606,74128153.0,286917566.0
793,1988,70000000,154112492.0,351500000.0,137875291,303547209.0,692330924.0
239,2005,80000000,128200012.0,260700012.0,95442650,152946861.0,311023751.0


In [414]:
#Formato dos dados
X_treino.shape, X_valid.shape, y_treino.shape, y_valid.shape
#Ajustando os dados de treino no modelo
modelo.fit(X_treino, y_treino.values.ravel())

((742, 7), (742, 7), (742, 1), (742, 1))

In [416]:
#Remodelando os dados de validação de X
p = modelo.predict(X_valid)
p=p.reshape(742,1)
y_valid.shape

(742, 1)

In [417]:
#Acurácia média dos dados previstos dentro do próprio banco de dados de treino de uma amostra aleatória
np.mean(y_valid == p)

0
binary    0.522911
dtype: float64

In [422]:
#Lista para armazenar os resultados
resultados = []

#Aplicação do método de Repeated Kfold, com 2 divisões de dados, 10 repetições e aleatorizados.
kf = RepeatedKFold(n_splits=2, n_repeats=10, random_state=20)

#Loop para impressão dos dados de treino e validação aleatórios, ajustando no modelo, prevendo os valores de teste e armazenando na lista de resultados
for linhas_treino, linhas_valid in kf.split(X):
    print("Treino:", linhas_treino.shape[0])
    print("Valid:", linhas_valid.shape[0])

    X_treino, X_valid = X.iloc[linhas_treino], X.iloc[linhas_valid]
    y_treino, y_valid = y.iloc[linhas_treino], y.iloc[linhas_valid]

    modelo = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=0)
    modelo.fit(X_treino, y_treino.values.ravel())

    p = modelo.predict(X_valid)
    pd.Series(p)
    p=p.reshape(742,1)

    acc = np.mean(y_valid == p)
    resultados.append(acc)
    print("Acurácia:", acc)
    print()

Treino: 742
Valid: 742
Acc: 0
binary    0.555256
dtype: float64

Treino: 742
Valid: 742
Acc: 0
binary    0.551213
dtype: float64

Treino: 742
Valid: 742
Acc: 0
binary    0.53504
dtype: float64

Treino: 742
Valid: 742
Acc: 0
binary    0.551213
dtype: float64

Treino: 742
Valid: 742
Acc: 0
binary    0.518868
dtype: float64

Treino: 742
Valid: 742
Acc: 0
binary    0.528302
dtype: float64

Treino: 742
Valid: 742
Acc: 0
binary    0.543127
dtype: float64

Treino: 742
Valid: 742
Acc: 0
binary    0.539084
dtype: float64

Treino: 742
Valid: 742
Acc: 0
binary    0.543127
dtype: float64

Treino: 742
Valid: 742
Acc: 0
binary    0.549865
dtype: float64

Treino: 742
Valid: 742
Acc: 0
binary    0.532345
dtype: float64

Treino: 742
Valid: 742
Acc: 0
binary    0.51752
dtype: float64

Treino: 742
Valid: 742
Acc: 0
binary    0.539084
dtype: float64

Treino: 742
Valid: 742
Acc: 0
binary    0.524259
dtype: float64

Treino: 742
Valid: 742
Acc: 0
binary    0.561995
dtype: float64

Treino: 742
Valid: 742
Acc:

In [434]:
#Aplicando a média dos resultados em %
np.mean(resultados)*100

53.68598382749325

In [440]:
h = pd.Series(h)
h.to_csv("mdb.csv", header=True)

Ou seja nosso modelo consegue prever 53,68% dos dados com precisão de filmes que passaram no teste de Bechdel. Para o aumento desta porcentagem, é interessante a aplicação de um "Grind Search" para outros modelos mais promissores. Também a refinação dos hiperparâmetros do modelo, onde se tornaria mais personalizado, porém mais trabalhoso.